In [2]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable, Type
import abc

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import torch.utils.data
from torch import Tensor
from sklearn.linear_model import RidgeClassifierCV
import xgboost as xgb

from models.ridge_ALOOCV import fit_ridge_ALOOCV
from models.sandwiched_least_squares import sandwiched_LS_dense, sandwiched_LS_diag, sandwiched_LS_scalar

In [3]:
# Make regression data X, y
N = 1000
N_test = 1000
D = 10
d = 3
X = torch.randn(N, D)
X_test = torch.randn(N_test, D)
w_true = torch.randn(D, d)
y = X @ w_true + torch.randn(N, d) * 0.1  # Adding some noise
y_test = X_test @ w_true + torch.randn(N_test, d) * 0.1  # Adding some noise

In [4]:
from models.models import GreedyRandFeatBoostRegression

#dense      
model = GreedyRandFeatBoostRegression(
     hidden_dim=128, 
     bottleneck_dim=128, 
     out_dim=d, 
     n_layers=5, 
     l2_reg=0.01, 
     feature_type="SWIM", 
     upscale="dense", 
     sandwich_solver="dense"
     )
model.fit(X, y)
out = model(X)
out_test = model(X_test)
print("out_test", out_test)
rmse = torch.sqrt(F.mse_loss(out, y))
rmse_test = torch.sqrt(F.mse_loss(out_test, y_test))
print("train rmse", rmse)
print("test rmse", rmse_test)

out_test tensor([[ 1.3051,  0.1853,  6.4380],
        [ 0.2428, -1.9974,  2.6660],
        [ 2.6222,  0.5761,  1.3131],
        ...,
        [-2.0423,  6.3859,  1.9665],
        [ 1.4709, -0.7372,  1.1072],
        [-1.3249, -2.3761, -3.7148]])
train rmse tensor(0.0936)
test rmse tensor(0.1042)


In [5]:
#diag
model = GreedyRandFeatBoostRegression(
     hidden_dim=128, 
     bottleneck_dim=128, 
     out_dim=d, 
     n_layers=5, 
     l2_reg=0.01, 
     feature_type="SWIM", 
     upscale="dense", 
     sandwich_solver="diag"
     )
model.fit(X, y)
out = model(X)
out_test = model(X_test)
print("out_test", out_test)
rmse = torch.sqrt(F.mse_loss(out, y))
rmse_test = torch.sqrt(F.mse_loss(out_test, y_test))
print("train rmse", rmse)
print("test rmse", rmse_test)

out_test tensor([[ 1.2558,  0.1604,  6.3965],
        [ 0.2072, -2.0072,  2.6722],
        [ 2.6354,  0.5783,  1.2984],
        ...,
        [-2.0159,  6.3566,  1.9361],
        [ 1.4744, -0.7536,  1.1107],
        [-1.3342, -2.3875, -3.7189]])
train rmse tensor(0.0987)
test rmse tensor(0.1019)


In [6]:
#scalar
model = GreedyRandFeatBoostRegression(
     hidden_dim=128, 
     bottleneck_dim=128, 
     out_dim=d, 
     n_layers=5, 
     l2_reg=0.01, 
     feature_type="SWIM", 
     upscale="dense", 
     sandwich_solver="scalar"
     )
model.fit(X, y)
out = model(X)
out_test = model(X_test)
print("out_test", out_test)
rmse = torch.sqrt(F.mse_loss(out, y))
rmse_test = torch.sqrt(F.mse_loss(out_test, y_test))
print("train rmse", rmse)
print("test rmse", rmse_test)

out_test tensor([[ 1.2725,  0.1859,  6.3949],
        [ 0.2064, -2.0042,  2.6664],
        [ 2.6210,  0.5817,  1.3088],
        ...,
        [-2.0131,  6.3515,  1.9350],
        [ 1.4768, -0.7548,  1.1051],
        [-1.3420, -2.3914, -3.7309]])
train rmse tensor(0.0994)
test rmse tensor(0.1017)


In [7]:
from models.models import GradientRandFeatBoostRegression
        
model = GradientRandFeatBoostRegression(
     hidden_dim=128, 
     bottleneck_dim=128, 
     out_dim=d, 
     n_layers=5, 
     feature_type="SWIM", 
     upscale="dense", 
     )
model.fit(X, y)
out = model(X)
out_test = model(X_test)
print("out_test", out_test)
rmse = torch.sqrt(F.mse_loss(out, y))
rmse_test = torch.sqrt(F.mse_loss(out_test, y_test))
print("train rmse", rmse)
print("test rmse", rmse_test)

out_test tensor([[ 1.2727,  0.1853,  6.3953],
        [ 0.2064, -2.0044,  2.6665],
        [ 2.6208,  0.5820,  1.3090],
        ...,
        [-2.0132,  6.3523,  1.9345],
        [ 1.4770, -0.7545,  1.1053],
        [-1.3421, -2.3920, -3.7299]])
train rmse tensor(0.0994)
test rmse tensor(0.1017)


# End2End

In [8]:
from models.models import End2EndMLPResNet

model = End2EndMLPResNet(
        in_dim=D,
        hidden_dim=32,
        bottleneck_dim=32,
        out_dim=d,
        n_blocks=3,
        loss = "mse",
        lr = 0.01,
        n_epochs = 30,
        end_lr_factor= 0.1,
        weight_decay = 0.001,
        batch_size = 64,
        )
model.fit(X, y)
out = model(X)
out_test = model(X_test)
print("out_test", out_test)
rmse = torch.sqrt(F.mse_loss(out, y))
rmse_test = torch.sqrt(F.mse_loss(out_test, y_test))
print("train rmse", rmse)
print("test rmse", rmse_test)

100%|██████████| 30/30 [00:02<00:00, 12.33it/s]

out_test tensor([[ 1.1037, -0.1023,  6.7920],
        [ 0.1250, -2.1999,  2.8912],
        [ 2.2738,  0.5330,  1.5370],
        ...,
        [-2.2105,  6.0674,  2.2782],
        [ 1.2726, -0.9610,  1.3701],
        [-1.2881, -2.2969, -3.9952]], grad_fn=<AddmmBackward0>)
train rmse tensor(0.1462, grad_fn=<SqrtBackward0>)
test rmse tensor(0.2552, grad_fn=<SqrtBackward0>)


# Next model